In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, model_from_json
from keras.optimizers import RMSprop,Adam,SGD
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D, UpSampling2D, Input
from keras.callbacks import CSVLogger
from livelossplot.keras import PlotLossesCallback
import efficientnet.keras as efn
import h5py
import numpy as np
import itertools
from keras.wrappers.scikit_learn import KerasClassifier
from cnn_utils import *
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

np.random.seed(1)

In [2]:
train_path = 'Spectograms_77_24_Xethrue_128x128/77_daily.hdf5'  # file path for the created .hdf5 file
trainset = h5py.File(train_path, "r")
train_path2 = 'Spectograms_77_24_Xethrue_128x128/24_daily.hdf5'  # file path for the created .hdf5 file
trainset2 = h5py.File(train_path2, "r")
train_path3 = 'Spectograms_77_24_Xethrue_128x128/XeThru_daily.hdf5'  # file path for the created .hdf5 file
trainset3 = h5py.File(train_path3, "r")

# valid_path = 'Spectograms_77_24_Xethrue_128x128/77_daily.hdf5'  # file path for the created .hdf5 file
# validset = h5py.File(valid_path, "r")
# valid_path2 = 'Spectograms_77_24_Xethrue_128x128/24_daily.hdf5'  # file path for the created .hdf5 file
# validset2 = h5py.File(valid_path2, "r")
# valid_path3 = 'Spectograms_77_24_Xethrue_128x128/XeThru_daily.hdf5'  # file path for the created .hdf5 file
# validset3 = h5py.File(valid_path3, "r")

test_path = 'Spectograms_77_24_Xethrue_128x128/77_daily.hdf5'  # file path for the created .hdf5 file
testset = h5py.File(test_path, "r")
test_path2 = 'Spectograms_77_24_Xethrue_128x128/24_daily.hdf5'  # file path for the created .hdf5 file
testset2 = h5py.File(test_path2, "r")
test_path3 = 'Spectograms_77_24_Xethrue_128x128/XeThru_daily.hdf5'  # file path for the created .hdf5 file
testset3 = h5py.File(test_path3, "r")

X_train_orig = np.array(trainset["train_img"])
Y_train_orig = np.array(trainset["train_labels"])
# X_valid_orig = np.array(validset["valid_img"])
# Y_valid_orig = np.array(validset["valid_labels"])
X_test_orig = np.array(testset["test_img"])
Y_test_orig = np.array(testset["test_labels"])
print('1. Radar''s Number of training samples: ', len(Y_train_orig))
# print('1. Radar''s Number of validation samples: ', len(X_valid_orig))
print('1. Radar''s Number of test samples: ', len(Y_test_orig))

X_train_orig2 = np.array(trainset2["train_img"])
Y_train_orig2 = np.array(trainset2["train_labels"])
# X_valid_orig2 = np.array(validset2["valid_img"])
# Y_valid_orig2 = np.array(validset2["valid_labels"])
X_test_orig2 = np.array(testset2["test_img"])
Y_test_orig2 = np.array(testset2["test_labels"])
print('2. Radar''s Number of training samples: ', len(X_train_orig2))
# print('2. Radar''s Number of validation samples: ', len(X_valid_orig2))
print('2. Radar''s Number of test samples: ', len(X_test_orig2))

X_train_orig3 = np.array(trainset3["train_img"])
Y_train_orig3 = np.array(trainset3["train_labels"])
# X_valid_orig3 = np.array(validset3["valid_img"])
# Y_valid_orig3 = np.array(validset3["valid_labels"])
X_test_orig3 = np.array(testset3["test_img"])
Y_test_orig3 = np.array(testset3["test_labels"])
print('3. Radar''s Number of training samples: ', len(X_train_orig3))
# print('3. Radar''s Number of validation samples: ', len(X_valid_orig3))
print('3. Radar''s Number of test samples: ', len(X_test_orig3))

trainset.close()
# validset.close()
testset.close()
trainset2.close()
# validset2.close()
testset2.close()
trainset3.close()
# validset3.close()
testset3.close()

1. Radars Number of training samples:  519
1. Radars Number of test samples:  130
2. Radars Number of training samples:  1695
2. Radars Number of test samples:  424
3. Radars Number of training samples:  614
3. Radars Number of test samples:  154


# Examine the shapes of data

In [3]:
X_train = X_train_orig/255.
# X_valid = X_valid_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 11).T
# Y_valid = convert_to_one_hot(Y_valid_orig, 11).T
Y_test = convert_to_one_hot(Y_test_orig, 11).T
print ("number of training examples = " + str(X_train.shape[0]))
# print ("number of validation examples = " + str(X_valid.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
# print ("X_valid shape: " + str(X_valid.shape))
# print ("Y_valid shape: " + str(Y_valid.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}

X_train2 = X_train_orig2/255.
# X_valid2 = X_valid_orig2/255.
X_test2 = X_test_orig2/255.
Y_train2 = convert_to_one_hot(Y_train_orig2, 11).T
# Y_valid2 = convert_to_one_hot(Y_valid_orig2, 11).T
Y_test2 = convert_to_one_hot(Y_test_orig2, 11).T
print ("\nnumber of training examples = " + str(X_train2.shape[0]))
# print ("number of validation examples = " + str(X_valid2.shape[0]))
print ("number of test examples = " + str(X_test2.shape[0]))
print ("X_train2 shape: " + str(X_train2.shape))
print ("Y_train2 shape: " + str(Y_train2.shape))
# print ("X_valid2 shape: " + str(X_valid2.shape))
# print ("Y_valid2 shape: " + str(Y_valid2.shape))
print ("X_test2 shape: " + str(X_test2.shape))
print ("Y_test2 shape: " + str(Y_test2.shape))

X_train3 = X_train_orig3/255.
# X_valid3 = X_valid_orig3/255.
X_test3 = X_test_orig3/255.
Y_train3 = convert_to_one_hot(Y_train_orig3, 11).T
# Y_valid3 = convert_to_one_hot(Y_valid_orig3, 11).T
Y_test3 = convert_to_one_hot(Y_test_orig3, 11).T
print ("\nnumber of training examples = " + str(X_train3.shape[0]))
# print ("number of validation examples = " + str(X_valid3.shape[0]))
print ("number of test examples = " + str(X_test3.shape[0]))
print ("X_train3 shape: " + str(X_train3.shape))
print ("Y_train3 shape: " + str(Y_train3.shape))
# print ("X_valid3 shape: " + str(X_valid3.shape))
# print ("Y_valid3 shape: " + str(Y_valid3.shape))
print ("X_test3 shape: " + str(X_test3.shape))
print ("Y_test3 shape: " + str(Y_test3.shape))

number of training examples = 519
number of test examples = 130
X_train shape: (519, 128, 128, 3)
Y_train shape: (519, 11)
X_test shape: (130, 128, 128, 3)
Y_test shape: (130, 11)

number of training examples = 1695
number of test examples = 424
X_train2 shape: (1695, 128, 128, 3)
Y_train2 shape: (1695, 11)
X_test2 shape: (424, 128, 128, 3)
Y_test2 shape: (424, 11)

number of training examples = 614
number of test examples = 154
X_train3 shape: (614, 128, 128, 3)
Y_train3 shape: (614, 11)
X_test3 shape: (154, 128, 128, 3)
Y_test3 shape: (154, 11)


# Select Dataset

In [4]:
# set flag: '0' for 77ghz, '1' for 24ghz, '2' for 10ghz
flag = 0

if flag==0:
    X_train = X_train
    Y_train = Y_train
#     X_valid = X_valid
#     Y_valid = Y_valid
    X_test = X_test
    Y_test = Y_test
    radar = 77
elif flag==1:
    X_train = X_train2
    Y_train = Y_train2
#     X_valid = X_valid2
#     Y_valid = Y_valid2
    X_test = X_test2
    Y_test = Y_test2
    radar = 24
elif flag==2: 
    X_train = X_train3
    Y_train = Y_train3
#     X_valid = X_valid3
#     Y_valid = Y_valid3
    X_test = X_test3
    Y_test = Y_test3
    radar = 10

In [30]:

def create_model(learn_rate = 0.001):
    model = Sequential()
    
    # Encoder
    model.add = Conv2D(8, (9, 9), activation='relu', padding='same') # 128x128x16
    model.add = Conv2D(8, (9, 9), activation='relu', padding='same')
    model.add = MaxPooling2D(pool_size=(2, 2)) # 64x64x16
    model.add = Conv2D(16, (3, 3), activation='relu', padding='same') # 64x64x32
    model.add = Conv2D(16, (3, 3), activation='relu', padding='same')
    model.add = MaxPooling2D(pool_size=(4, 4)) # 16x16x32
    model.add = Conv2D(32, (3, 3), activation='relu', padding='same') # 16x16x64
    model.add = Conv2D(32, (3, 3), activation='relu', padding='same')
    model.add = MaxPooling2D(pool_size=(4, 4)) # 4x4x64  (small and thick)
    
    # Decoder
    model.add = Conv2D(32, (3, 3), activation='relu', padding='same') # 16x16x32
    model.add = Conv2D(32, (3, 3), activation='relu', padding='same')
    model.add = UpSampling2D((4,4)) #  16x16x64
    model.add = Conv2D(16, (3, 3), activation='relu', padding='same') # 16x16x16
    model.add = Conv2D(16, (3, 3), activation='relu', padding='same')
    model.add = UpSampling2D((4,4)) # 64x64x16
    model.add = Conv2D(8, (9, 9), activation='relu', padding='same') # 64x64x16
    model.add = Conv2D(8, (9, 9), activation='relu', padding='same')
    model.add = UpSampling2D((2,2)) #  128x128x16
    model.add = Conv2D(3, (3, 3), activation='sigmoid', padding='same') # 128 x 128 x 3 / Conv2D(1, (3, 3)
    
    optimizer = Adam(lr = learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
 

## Grid Search

In [33]:
epochs = [100,150,250,300]
batch_size = [8, 16, 32, 64 ,128]
learn_rate = [0.001, 0.0005, 0.0001, 0.0007, 0.0003]
param_grid = dict(epochs=epochs,batch_size=batch_size,learn_rate=learn_rate)
# create model
inChannel = 3
x, y = 128, 128
input_img = Input(shape = (x, y, inChannel))
model = KerasClassifier(build_fn=create_model, verbose=1)
model = Model(input_img,create_model(input_img), verbose=1)
model2 = Model(input_img, decoder(encoder2(input_img)))
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3) # use all the cores (n_ jobs =-1), or set to 1 if err
grid_result = grid.fit(X_train, X_train)

ValueError: Tensor-typed variable initializers must either be wrapped in an init_scope or callable (e.g., `tf.Variable(lambda : tf.truncated_normal([10, 40]))`) when building functions. Please file a feature request if this restriction inconveniences you.